In [1]:
!apt-get update                                                                          # Update Package 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null                                  # install Java Development Kit (important!!! cant run Spark with out this)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz # install Spark 3.1.2
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz                                                  # Unzip  Spark 3.1.2
!pip install -q findspark==1.3.0                                                         # install Package Python for connect with Spark 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import pandas as pd

In [4]:
!pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 78 kB/s 
     |████████████████████████████████| 198 kB 66.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=13d7d76df97b2c5c0b47422c8852a866b2f2198a0f1229487576e0410aaae3f5
  Stored in directory: /root/.cache/pip/wheels/df/88/9e/58ef1f74892fef590330ca0830b5b6d995ba29b44f977b3926
Successfully built pyspark


In [5]:
# Create Spark Session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
#  Python Version
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=15, releaselevel='final', serial=0)

In [7]:
# Download Data File !wget

!wget https://maven-datasets.s3.amazonaws.com/Wine+Tasting/winemag-data-130k-v2.csv.zip

--2022-12-06 13:04:04--  https://maven-datasets.s3.amazonaws.com/Wine+Tasting/winemag-data-130k-v2.csv.zip
Resolving maven-datasets.s3.amazonaws.com (maven-datasets.s3.amazonaws.com)... 52.216.130.171, 52.216.63.1, 52.216.239.51, ...
Connecting to maven-datasets.s3.amazonaws.com (maven-datasets.s3.amazonaws.com)|52.216.130.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16934504 (16M) [application/zip]
Saving to: ‘winemag-data-130k-v2.csv.zip’

winemag-data-130k-v 100%[===================>]  16.15M  60.0MB/s    in 0.3s    

2022-12-06 13:04:04 (60.0 MB/s) - ‘winemag-data-130k-v2.csv.zip’ saved [16934504/16934504]



In [8]:
!unzip winemag-data-130k-v2.csv.zip

Archive:  winemag-data-130k-v2.csv.zip
  inflating: winemag-data-130k-v2.csv  
  inflating: __MACOSX/._winemag-data-130k-v2.csv  


In [9]:
dt = spark.read.csv('/content/winemag-data-130k-v2.csv', header = True, inferSchema = True, )

In [10]:
# check column
dt

DataFrame[id: string, country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, taster_name: string, taster_twitter_handle: string, title: string, variety: string, winery: string]

In [11]:
dt.dtypes

[('id', 'string'),
 ('country', 'string'),
 ('description', 'string'),
 ('designation', 'string'),
 ('points', 'string'),
 ('price', 'string'),
 ('province', 'string'),
 ('region_1', 'string'),
 ('region_2', 'string'),
 ('taster_name', 'string'),
 ('taster_twitter_handle', 'string'),
 ('title', 'string'),
 ('variety', 'string'),
 ('winery', 'string')]

In [12]:
dt.printSchema()

root
 |-- id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [13]:
dt.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
| id|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|

In [14]:
print((dt.count(), len(dt.columns)))

(129975, 14)


In [15]:
dt.dropDuplicates().count()


129975

In [16]:
dt.summary().show()

+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|summary|               id|             country|         description|         designation|              points|               price|            province|            region_1|            region_2|         taster_name|taster_twitter_handle|               title|             variety|              winery|
+-------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+
|  count|           129975|              129912|              129974|               92513|    

In [17]:
dt.summary("count").show()

+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+
|summary|    id|country|description|designation|points| price|province|region_1|region_2|taster_name|taster_twitter_handle| title|variety|winery|
+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+
|  count|129975| 129912|     129974|      92513|129966|120976|  129909|  108734|   50521|     103726|                98762|129965| 129963|129963|
+-------+------+-------+-----------+-----------+------+------+--------+--------+--------+-----------+---------------------+------+-------+------+



In [18]:
dt = dt.drop('region_1','region_2','region_')
dt.show()

+---+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
| id|  country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|        Roger Voss|           @vossroger|Quinta dos Avidag...|    Portuguese Red|Quinta dos Avidagos|
|  2|       US|Tart and snappy, ...|              

In [19]:
dt.where( dt.points.isNull()).show()

+--------------------+--------------------+--------------------+--------------------+------+-----+-------------+-----------+---------------------+------+--------------------+------------+
|                  id|             country|         description|         designation|points|price|     province|taster_name|taster_twitter_handle| title|             variety|      winery|
+--------------------+--------------------+--------------------+--------------------+------+-----+-------------+-----------+---------------------+------+--------------------+------------+
|               18881|        South Africa|Notes of cocoa an...|Grand Classique C...|  null| null|         null|       null|                 null|  null|                null|        null|
|Merlot-Cabernet F...|                  90|                  20|               Paarl|  null| null|Lauren Buzzeo|       null|                 null|  null|                null|        null|
|Merlot-Cabernet F...|Bordeaux-style Re...|         Glen Car

In [20]:
dt.groupBy('taster_name').count().orderBy('count', ascending = False).show(38)

+--------------------+-----+
|         taster_name|count|
+--------------------+-----+
|                null|26249|
|          Roger Voss|25513|
|   Michael Schachner|15133|
|       Kerin O’Keefe|10776|
|      Virginie Boone| 9537|
|        Paul Gregutt| 9529|
|       Matt Kettmann| 6332|
|      Joe Czerwinski| 5139|
|    Sean P. Sullivan| 4966|
|  Anna Lee C. Iijima| 4415|
|          Jim Gordon| 4177|
|    Anne Krebiehl MW| 3683|
|       Lauren Buzzeo| 1832|
|     Susan Kostrzewa| 1085|
|       Mike DeSimone|  514|
|        Jeff Jenssen|  491|
|  Alexander Peartree|  415|
|        Carrie Dykes|  139|
|         Fiona Adams|   27|
|   Christina Pickard|    6|
|                  89|    2|
|   Chambolle-Musigny|    1|
| there's remarkab...|    1|
|                Etna|    1|
|                  95|    1|
| mocha and cinnam...|    1|
|                  84|    1|
| almost severe el...|    1|
| but you can cert...|    1|
|                  25|    1|
|                  86|    1|
|         Napa

In [21]:
dt.where('''country in ('90',
                        ' fine and extremely polished; hold for 10 years."',
                        ' marked by tart citrus flavors that make it versatile with a wide range of foods."',
                        'Bordeaux-style Red Blend')''').show()

+--------------------+--------------------+-----------+-----------+------+----------+-------------+-----------+---------------------+------------+-----------+------+
|                  id|             country|description|designation|points|     price|     province|taster_name|taster_twitter_handle|       title|    variety|winery|
+--------------------+--------------------+-----------+-----------+------+----------+-------------+-----------+---------------------+------------+-----------+------+
|Merlot-Cabernet F...|                  90|         20|      Paarl|  null|      null|Lauren Buzzeo|       null|                 null|        null|       null|  null|
|Merlot-Cabernet F...|Bordeaux-style Re...|Glen Carlou|       null|  null|      null|         null|       null|                 null|        null|       null|  null|
|             elegant| fine and extreme...|     Ravera|         92|    71|  Piedmont|       Barolo|       null| Elvio Cogno 2008 ...|    Nebbiolo|Elvio Cogno|  null|
|   

In [22]:
dt_c = dt.groupBy('taster_name').count().orderBy('count', ascending = False)
dt_c = dt_c.where('count == 1')
dt_c.show()

+--------------------+-----+
|         taster_name|count|
+--------------------+-----+
|   Chambolle-Musigny|    1|
|                Etna|    1|
| there's remarkab...|    1|
|                  95|    1|
|                  25|    1|
|                  86|    1|
| mocha and cinnam...|    1|
|                  84|    1|
|         Napa Valley|    1|
| almost severe el...|    1|
| but you can cert...|    1|
|       Central Coast|    1|
|          California|    1|
|        Finger Lakes|    1|
|Edition Chremisa ...|    1|
+--------------------+-----+



In [23]:
from pyspark.sql import functions as F, Window as W


In [24]:
window = W.partitionBy("taster_name")

dt_name = dt.withColumn("count", F.count("taster_name").over(window))\
              .filter(F.col("count") > 2).drop("count")
dt_name.groupBy('taster_name').count().orderBy('count', ascending = False).show(38)

+------------------+-----+
|       taster_name|count|
+------------------+-----+
|        Roger Voss|25513|
| Michael Schachner|15133|
|     Kerin O’Keefe|10776|
|    Virginie Boone| 9537|
|      Paul Gregutt| 9529|
|     Matt Kettmann| 6332|
|    Joe Czerwinski| 5139|
|  Sean P. Sullivan| 4966|
|Anna Lee C. Iijima| 4415|
|        Jim Gordon| 4177|
|  Anne Krebiehl MW| 3683|
|     Lauren Buzzeo| 1832|
|   Susan Kostrzewa| 1085|
|     Mike DeSimone|  514|
|      Jeff Jenssen|  491|
|Alexander Peartree|  415|
|      Carrie Dykes|  139|
|       Fiona Adams|   27|
| Christina Pickard|    6|
+------------------+-----+



In [25]:
dt = dt_name
dt.groupBy('taster_name').count().orderBy('count', ascending = False).show()

+------------------+-----+
|       taster_name|count|
+------------------+-----+
|        Roger Voss|25513|
| Michael Schachner|15133|
|     Kerin O’Keefe|10776|
|    Virginie Boone| 9537|
|      Paul Gregutt| 9529|
|     Matt Kettmann| 6332|
|    Joe Czerwinski| 5139|
|  Sean P. Sullivan| 4966|
|Anna Lee C. Iijima| 4415|
|        Jim Gordon| 4177|
|  Anne Krebiehl MW| 3683|
|     Lauren Buzzeo| 1832|
|   Susan Kostrzewa| 1085|
|     Mike DeSimone|  514|
|      Jeff Jenssen|  491|
|Alexander Peartree|  415|
|      Carrie Dykes|  139|
|       Fiona Adams|   27|
| Christina Pickard|    6|
+------------------+-----+



In [26]:
dt.select("country").distinct().count()

44

In [27]:
dt.select("country").distinct().sort("country").show( 48, False )

+----------------------+
|country               |
+----------------------+
|null                  |
|Argentina             |
|Armenia               |
|Australia             |
|Austria               |
|Bosnia and Herzegovina|
|Brazil                |
|Bulgaria              |
|Canada                |
|Chile                 |
|China                 |
|Croatia               |
|Cyprus                |
|Czech Republic        |
|Egypt                 |
|England               |
|France                |
|Georgia               |
|Germany               |
|Greece                |
|Hungary               |
|India                 |
|Israel                |
|Italy                 |
|Lebanon               |
|Luxembourg            |
|Macedonia             |
|Mexico                |
|Moldova               |
|Morocco               |
|New Zealand           |
|Peru                  |
|Portugal              |
|Romania               |
|Serbia                |
|Slovakia              |
|Slovenia              |


In [28]:
dt.groupBy('country').count().orderBy('count', ascending = False).show(48, False)

+----------------------+-----+
|country               |count|
+----------------------+-----+
|US                    |37726|
|France                |21827|
|Italy                 |11041|
|Spain                 |6580 |
|Portugal              |5686 |
|Chile                 |4361 |
|Argentina             |3797 |
|Austria               |3334 |
|Germany               |2134 |
|Australia             |2037 |
|South Africa          |1326 |
|New Zealand           |1306 |
|Israel                |500  |
|Greece                |466  |
|Canada                |256  |
|Hungary               |145  |
|Bulgaria              |141  |
|Romania               |120  |
|Uruguay               |109  |
|Turkey                |90   |
|Slovenia              |87   |
|Georgia               |86   |
|England               |74   |
|Croatia               |73   |
|Mexico                |65   |
|null                  |62   |
|Moldova               |59   |
|Brazil                |52   |
|Lebanon               |35   |
|Morocco

In [29]:
window_c = W.partitionBy("country")

dt_country = dt.withColumn("count", F.count("country").over(window_c))\
              .filter(F.col("count") > 1).drop("count")
dt_country.groupBy('country').count().orderBy('count', ascending = False).show(48)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                  US|37726|
|              France|21827|
|               Italy|11041|
|               Spain| 6580|
|            Portugal| 5686|
|               Chile| 4361|
|           Argentina| 3797|
|             Austria| 3334|
|             Germany| 2134|
|           Australia| 2037|
|        South Africa| 1326|
|         New Zealand| 1306|
|              Israel|  500|
|              Greece|  466|
|              Canada|  256|
|             Hungary|  145|
|            Bulgaria|  141|
|             Romania|  120|
|             Uruguay|  109|
|              Turkey|   90|
|            Slovenia|   87|
|             Georgia|   86|
|             England|   74|
|             Croatia|   73|
|              Mexico|   65|
|             Moldova|   59|
|              Brazil|   52|
|             Lebanon|   35|
|             Morocco|   28|
|                Peru|   16|
|             Ukraine|   14|
|      Czech R

In [30]:
dt_cnt = dt_country.groupBy('country').count().orderBy('count', ascending = False)
dt_cnt.show(44)

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                  US|37726|
|              France|21827|
|               Italy|11041|
|               Spain| 6580|
|            Portugal| 5686|
|               Chile| 4361|
|           Argentina| 3797|
|             Austria| 3334|
|             Germany| 2134|
|           Australia| 2037|
|        South Africa| 1326|
|         New Zealand| 1306|
|              Israel|  500|
|              Greece|  466|
|              Canada|  256|
|             Hungary|  145|
|            Bulgaria|  141|
|             Romania|  120|
|             Uruguay|  109|
|              Turkey|   90|
|            Slovenia|   87|
|             Georgia|   86|
|             England|   74|
|             Croatia|   73|
|              Mexico|   65|
|             Moldova|   59|
|              Brazil|   52|
|             Lebanon|   35|
|             Morocco|   28|
|                Peru|   16|
|             Ukraine|   14|
|             

In [31]:
dt_null_c = dt.filter("country IS NULL")
dt_null_c.show()

+------+-------+--------------------+--------------------+------+-----+--------+----------------+---------------------+--------------------+--------------------+-----------------+
|    id|country|         description|         designation|points|price|province|     taster_name|taster_twitter_handle|               title|             variety|           winery|
+------+-------+--------------------+--------------------+------+-----+--------+----------------+---------------------+--------------------+--------------------+-----------------+
|  3131|   null|Soft, fruity and ...|            Partager|    83| null|    null|      Roger Voss|           @vossroger|Barton & Guestier...|           Red Blend|Barton & Guestier|
| 11348|   null|Light and fruity,...|            Partager|    82| null|    null|      Roger Voss|           @vossroger|Barton & Guestier...|         White Blend|Barton & Guestier|
|  9509|   null|This mouthwaterin...|Theopetra Malagou...|    92|   28|    null| Susan Kostrzewa|   

In [32]:
dt.show()

+---+------------+--------------------+--------------------+------+-----+--------------------+-----------+---------------------+--------------------+--------------------+--------------------+
| id|     country|         description|         designation|points|price|            province|taster_name|taster_twitter_handle|               title|             variety|              winery|
+---+------------+--------------------+--------------------+------+-----+--------------------+-----------+---------------------+--------------------+--------------------+--------------------+
|  1|    Portugal|This is ripe and ...|            Avidagos|    87|   15|               Douro| Roger Voss|           @vossroger|Quinta dos Avidag...|      Portuguese Red| Quinta dos Avidagos|
|  7|      France|This dry and rest...|                null|    87|   24|              Alsace| Roger Voss|           @vossroger|Trimbach 2012 Gew...|      Gewürztraminer|            Trimbach|
|  9|      France|This has great de...| 

In [33]:
dt.groupBy('points').count().orderBy('points', ascending = True).show(100)

+------+-----+
|points|count|
+------+-----+
|   100|   14|
|    80|  275|
|    81|  433|
|    82| 1054|
|    83| 2112|
|    84| 4394|
|    85| 7132|
|    86| 9578|
|    87|12870|
|    88|14355|
|    89|10656|
|    90|12942|
|    91| 9765|
|    92| 7832|
|    93| 5405|
|    94| 3018|
|    95| 1221|
|    96|  405|
|    97|  166|
|    98|   61|
|    99|   21|
+------+-----+



In [34]:
window_points = W.partitionBy("points")

dt_point = dt.withColumn("count", F.count("points").over(window_points))\
              .filter(F.col("count") > 3).drop("count")
dt_point.groupBy('points').count().orderBy('count', ascending = False).show(48)


+------+-----+
|points|count|
+------+-----+
|    88|14355|
|    90|12942|
|    87|12870|
|    89|10656|
|    91| 9765|
|    86| 9578|
|    92| 7832|
|    85| 7132|
|    93| 5405|
|    84| 4394|
|    94| 3018|
|    83| 2112|
|    95| 1221|
|    82| 1054|
|    81|  433|
|    96|  405|
|    80|  275|
|    97|  166|
|    98|   61|
|    99|   21|
|   100|   14|
+------+-----+



In [35]:
dt.filter("variety IS NULL").count()

0

In [36]:
dt.groupBy('variety').count().orderBy('count', ascending = False).show(20)

+--------------------+-----+
|             variety|count|
+--------------------+-----+
|          Pinot Noir|10317|
|          Chardonnay| 8942|
|           Red Blend| 7108|
|Bordeaux-style Re...| 6389|
|  Cabernet Sauvignon| 6271|
|            Riesling| 4930|
|     Sauvignon Blanc| 4018|
|                Rosé| 3315|
|               Syrah| 3140|
|              Malbec| 2583|
|      Portuguese Red| 2466|
|              Merlot| 2099|
|          Sangiovese| 1997|
|            Nebbiolo| 1981|
|         White Blend| 1778|
|         Tempranillo| 1755|
|     Sparkling Blend| 1730|
|           Zinfandel| 1580|
|          Pinot Gris| 1338|
|    Grüner Veltliner| 1335|
+--------------------+-----+
only showing top 20 rows



In [37]:
window_variety= W.partitionBy("variety")

dt_variety = dt.withColumn("count", F.count("variety").over(window_variety))\
              .filter(F.col("count") > 20).drop("count")
dt_variety.groupBy('variety').count().orderBy('count', ascending = True).show()


+--------------------+-----+
|             variety|count|
+--------------------+-----+
|           Colombard|   21|
|    Monastrell-Syrah|   21|
|            Savagnin|   22|
|              Kerner|   22|
|       Verdejo-Viura|   22|
|             Schiava|   23|
|           Scheurebe|   23|
|          Gamay Noir|   23|
|               Bobal|   23|
|           Teroldego|   23|
|      Muskat Ottonel|   23|
|             Macabeo|   23|
|            Jacquère|   24|
|            Pecorino|   24|
|      Welschriesling|   24|
|              Claret|   24|
|              Tokaji|   24|
|Cabernet Sauvigno...|   25|
|        Gros Manseng|   25|
|          Dornfelder|   26|
+--------------------+-----+
only showing top 20 rows



In [38]:
dt = dt_variety
dt.groupBy('variety').count().orderBy('count', ascending = True).show(50)

+--------------------+-----+
|             variety|count|
+--------------------+-----+
|    Monastrell-Syrah|   21|
|           Colombard|   21|
|              Kerner|   22|
|            Savagnin|   22|
|       Verdejo-Viura|   22|
|          Gamay Noir|   23|
|             Macabeo|   23|
|           Scheurebe|   23|
|               Bobal|   23|
|             Schiava|   23|
|      Muskat Ottonel|   23|
|           Teroldego|   23|
|              Tokaji|   24|
|            Pecorino|   24|
|      Welschriesling|   24|
|            Jacquère|   24|
|              Claret|   24|
|Cabernet Sauvigno...|   25|
|        Gros Manseng|   25|
|         Trincadeira|   26|
|          Dornfelder|   26|
|            Frappato|   26|
|          Rotgipfler|   26|
|           Auxerrois|   27|
|            Malvasia|   27|
|           Carignane|   27|
|             Aligoté|   27|
|Touriga Nacional-...|   28|
|          Carricante|   29|
|            Loureiro|   29|
| Chardonnay-Viognier|   29|
|          Fum

In [39]:
dt.show(20)

+------+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
|    id|  country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|             variety|              winery|
+------+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
| 15995|   France|This simple, frui...|                null|    86|   12|         Bordeaux|        Roger Voss|           @vossroger|Château Saint-Flo...|Sauvignon Blanc-S...|Château Saint-Flo...|
|116181|       US|Whispers of smoke...|               Haven|    90|   35|         New York|Anna Lee C. Iijima|                 null|Shinn Estate 2010...|Sauvignon Blanc-S...|        Shinn Estate|
| 17501|       US|Th

In [40]:
dt.printSchema()

root
 |-- id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [41]:
from pyspark.sql.types import IntegerType

In [42]:
dt = dt.withColumn("price", dt["price"].cast(IntegerType()))
dt = dt.withColumn("points", dt["points"].cast(IntegerType()))

In [43]:
dt.printSchema()

root
 |-- id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [44]:
import pandas as pd

In [45]:
## chage to pandas data frame
dt_clean_pd = dt.toPandas()

In [46]:
dt_clean_pd.head()

,id,country,description,designation,points,price,province,taster_name,taster_twitter_handle,title,variety,winery
0,15995,France,"This simple, fruity wine has both crisp, citru...",None,86,12.0,Bordeaux,Roger Voss,@vossroger,Château Saint-Florian 2016 Bordeaux Blanc,Sauvignon Blanc-Semillon,Château Saint-Florian
1,116181,US,Whispers of smoke and fresh herb are a counter...,Haven,90,35.0,New York,Anna Lee C. Iijima,None,Shinn Estate 2010 Haven Sauvignon Blanc-Semill...,Sauvignon Blanc-Semillon,Shinn Estate
2,17501,US,"The two varieties show themselves in turn, wit...",SBS,89,23.0,Washington,Sean P. Sullivan,@wawinereport,Cadaretta 2014 SBS Sauvignon Blanc-Semillon (C...,Sauvignon Blanc-Semillon,Cadaretta
3,119827,US,"Sauvignon Blanc makes up 89% of this wine, wit...",SBS,88,23.0,Washington,Sean P. Sullivan,@wawinereport,Cadaretta 2015 SBS Sauvignon Blanc-Semillon (C...,Sauvignon Blanc-Semillon,Cadaretta
4,106843,US,"Made from Sémillon and Sauvignon Blanc, this w...",Il Passito,94,89.0,California,Jim Gordon,@gordone_cellars,Castello di Amorosa 2010 Il Passito Sauvignon ...,Sauvignon Blanc-Semillon,Castello di Amorosa


In [47]:
dt.createOrReplaceTempView("data")
dt_sql = spark.sql("SELECT * FROM data")
dt_sql.show()

+------+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
|    id|  country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|             variety|              winery|
+------+---------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
| 15995|   France|This simple, frui...|                null|    86|   12|         Bordeaux|        Roger Voss|           @vossroger|Château Saint-Flo...|Sauvignon Blanc-S...|Château Saint-Flo...|
|116181|       US|Whispers of smoke...|               Haven|    90|   35|         New York|Anna Lee C. Iijima|                 null|Shinn Estate 2010...|Sauvignon Blanc-S...|        Shinn Estate|
| 17501|       US|Th

In [48]:
avg_by_country = dt.groupBy('country').agg(F.avg('price'),F.avg('points'))
avg_by_country.show(50)


+--------------+------------------+-----------------+
|       country|        avg(price)|      avg(points)|
+--------------+------------------+-----------------+
|        Turkey|27.974358974358974|88.05128205128206|
|       Germany| 42.69394673123487|89.87821122740247|
|        France| 41.50162356488461|88.87293767718548|
|        Greece|22.854922279792746|87.19693094629156|
|          null|24.954545454545453|88.48936170212765|
|      Slovakia|              16.0|             87.0|
|     Argentina| 24.22760686087667|86.68998923573736|
|          Peru|           18.0625|          83.5625|
|         India|13.333333333333334|90.22222222222223|
|         China|              18.0|             89.0|
|         Chile|            20.864| 86.5099907063197|
|       Croatia|20.551724137931036|86.51612903225806|
|         Italy| 42.14536135315223|88.87079496335275|
|         Spain|28.546922952673235|87.35605939645538|
|            US| 36.27972254460069|88.98663732961253|
|       Morocco|19.703703703

In [49]:
avg_by_country = avg_by_country.withColumn('avg(price)', F.round('avg(price)',2))
avg_by_country = avg_by_country.withColumn('avg(points)', F.round('avg(points)',2))
avg_by_country = avg_by_country.withColumnRenamed('avg(points)', 'AVG_POINT')
avg_by_country = avg_by_country.withColumnRenamed('avg(price)', 'AVG_PRICE')

avg_by_country.show()

+---------+---------+---------+
|  country|AVG_PRICE|AVG_POINT|
+---------+---------+---------+
|   Turkey|    27.97|    88.05|
|  Germany|    42.69|    89.88|
|   France|     41.5|    88.87|
|   Greece|    22.85|     87.2|
|     null|    24.95|    88.49|
| Slovakia|     16.0|     87.0|
|Argentina|    24.23|    86.69|
|     Peru|    18.06|    83.56|
|    India|    13.33|    90.22|
|    China|     18.0|     89.0|
|    Chile|    20.86|    86.51|
|  Croatia|    20.55|    86.52|
|    Italy|    42.15|    88.87|
|    Spain|    28.55|    87.36|
|       US|    36.28|    88.99|
|  Morocco|     19.7|    88.56|
|  Ukraine|     9.25|    83.83|
|   Israel|    31.88|    88.51|
|   Cyprus|    15.25|     87.0|
|  Uruguay|    27.25|    86.72|
+---------+---------+---------+
only showing top 20 rows



In [50]:
dt_cnt.show()

+------------+-----+
|     country|count|
+------------+-----+
|          US|37726|
|      France|21827|
|       Italy|11041|
|       Spain| 6580|
|    Portugal| 5686|
|       Chile| 4361|
|   Argentina| 3797|
|     Austria| 3334|
|     Germany| 2134|
|   Australia| 2037|
|South Africa| 1326|
| New Zealand| 1306|
|      Israel|  500|
|      Greece|  466|
|      Canada|  256|
|     Hungary|  145|
|    Bulgaria|  141|
|     Romania|  120|
|     Uruguay|  109|
|      Turkey|   90|
+------------+-----+
only showing top 20 rows



In [51]:
dt_avg = avg_by_country.join(dt_cnt,['country']).orderBy('AVG_POINT','AVG_PRICE', ascending=False)
dt_avg.show(50)

+--------------+---------+---------+-----+
|       country|AVG_PRICE|AVG_POINT|count|
+--------------+---------+---------+-----+
|       England|    51.68|    91.58|   74|
|         India|    13.33|    90.22|    9|
|       Austria|    30.73|    90.11| 3334|
|       Germany|    42.69|    89.88| 2134|
|       Hungary|    42.51|    89.42|  145|
|        Canada|    35.42|    89.35|  256|
|            US|    36.28|    88.99|37726|
|         Italy|    42.15|    88.87|11041|
|        France|     41.5|    88.87|21827|
|     Australia|    37.16|    88.76| 2037|
|    Luxembourg|    23.33|    88.67|    6|
|       Morocco|     19.7|    88.56|   28|
|        Israel|    31.88|    88.51|  500|
|   New Zealand|     27.3|    88.33| 1306|
|      Portugal|    25.94|    88.25| 5686|
|  South Africa|    25.14|    88.18| 1326|
|        Turkey|    27.97|    88.05|   90|
|      Slovenia|    24.46|    88.03|   87|
|   Switzerland|    107.8|     88.0|    7|
|        Serbia|     31.0|     88.0|   12|
|       Arm

In [52]:
dt.orderBy('id').show()

+------+--------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
|    id| country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|             variety|              winery|
+------+--------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
|     0|   Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|         White Blend|             Nicosia|
|     1|Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|        Roger Voss|           @vossroger|Quinta dos Avidag...|      Portuguese Red| Quinta dos Avidagos|
|    10|      US|Soft, su

In [53]:
w_1 = W.orderBy("id")
result = dt.withColumn("ID", F.row_number().over(w_1))
result.show()

+---+--------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
| ID| country|         description|         designation|points|price|         province|       taster_name|taster_twitter_handle|               title|             variety|              winery|
+---+--------+--------------------+--------------------+------+-----+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+
|  1|   Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|         White Blend|             Nicosia|
|  2|Portugal|This is ripe and ...|            Avidagos|    87|   15|            Douro|        Roger Voss|           @vossroger|Quinta dos Avidag...|      Portuguese Red| Quinta dos Avidagos|
|  3|      US|Soft, supple plum...|     

In [54]:
result.summary("count").show()

+-------+------+-------+-----------+-----------+------+-----+--------+-----------+---------------------+------+-------+------+
|summary|    ID|country|description|designation|points|price|province|taster_name|taster_twitter_handle| title|variety|winery|
+-------+------+-------+-----------+-----------+------+-----+--------+-----------+---------------------+------+-------+------+
|  count|101305| 101258|     101305|      73224|101305|94198|  101258|     101305|                96495|101305| 101305|101305|
+-------+------+-------+-----------+-----------+------+-----+--------+-----------+---------------------+------+-------+------+



In [55]:
result.coalesce(1).write.csv('wine_data_clean.csv', header = True)
print('file save successfully')

file save successfully


In [56]:
dt_avg.coalesce(1).write.csv('wine_avg.csv', header = True)
print('file save successfully')

file save successfully


In [58]:
read_wine_data = spark.read.csv('/content/wine_data_clean.csv/part-00000-abd995f0-0083-4ea4-b0d3-bed8b02fa0b9-c000.csv', header = True, inferSchema = True)
read_wine_data.show()

AnalysisException: ignored

In [ ]:
read_avg_data = spark.read.csv('/content/wine_avg.csv/part-00000-857a011c-fad7-4fdd-bf36-dec73ee07376-c000.csv', header = True, inferSchema = True)
read_avg_data.show()

In [ ]:
!pip install gazpacho

In [60]:
from gazpacho import Soup
from requests import get

In [61]:
url = "https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=THB"

In [65]:
resp = get(url)
resp.status_code ## if return code 200 means o, runfine and website can be access

200

In [66]:
exc_rate = Soup(resp.text)

In [67]:
th_rate = exc_rate.find('p', {'class' : 'result__BigRate-sc-1bsijpp-1 iGrAod'}).strip() 
th_rate

'35.104183 Thai Baht'

In [68]:
thb_usd = round(float(th_rate.split()[0]),2)
unit = th_rate.split()[2]
currency = th_rate.split()[1]
print('the unit rate for 1 USD:', thb_usd, currency, unit)

the unit rate for 1 USD: 35.1 Thai Baht


In [69]:
thb_usd

35.1

In [75]:
data = [thb_usd]
currency_rate = ['THB_USD']
df_rate = spark.createDataFrame(data = [data], schema = currency_rate)
df_rate.show()

+-------+
|THB_USD|
+-------+
|   35.1|
+-------+



In [78]:
dt_avg_TH = dt_avg.join(df_rate)
dt_avg_TH.show()

+---------+---------+---------+-----+-------+
|  country|AVG_PRICE|AVG_POINT|count|THB_USD|
+---------+---------+---------+-----+-------+
|   Turkey|    27.97|    88.05|   90|   35.1|
|  Germany|    42.69|    89.88| 2134|   35.1|
|   France|     41.5|    88.87|21827|   35.1|
|   Greece|    22.85|     87.2|  466|   35.1|
|Argentina|    24.23|    86.69| 3797|   35.1|
|     Peru|    18.06|    83.56|   16|   35.1|
|    India|    13.33|    90.22|    9|   35.1|
|    Chile|    20.86|    86.51| 4361|   35.1|
|  Croatia|    20.55|    86.52|   73|   35.1|
|    Italy|    42.15|    88.87|11041|   35.1|
|    Spain|    28.55|    87.36| 6580|   35.1|
|       US|    36.28|    88.99|37726|   35.1|
|  Morocco|     19.7|    88.56|   28|   35.1|
|  Ukraine|     9.25|    83.83|   14|   35.1|
|   Israel|    31.88|    88.51|  500|   35.1|
|   Cyprus|    15.25|     87.0|   11|   35.1|
|  Uruguay|    27.25|    86.72|  109|   35.1|
|   Mexico|     27.9|     85.1|   65|   35.1|
|  Georgia|    19.04|    87.76|   

In [89]:
dt_avg_TH = dt_avg_TH.withColumn('THB_USD',F.col('THB_USD')*F.col('AVG_PRICE'))
dt_avg_TH = dt_avg_TH.withColumn('THB_USD', F.round('THB_USD',2))
dt_avg_TH.show()

+---------+---------+---------+-----+-------+
|  country|AVG_PRICE|AVG_POINT|count|THB_USD|
+---------+---------+---------+-----+-------+
|   Turkey|    27.97|    88.05|   90| 981.75|
|  Germany|    42.69|    89.88| 2134|1498.42|
|   France|     41.5|    88.87|21827|1456.65|
|   Greece|    22.85|     87.2|  466| 802.04|
|Argentina|    24.23|    86.69| 3797| 850.47|
|     Peru|    18.06|    83.56|   16| 633.91|
|    India|    13.33|    90.22|    9| 467.88|
|    Chile|    20.86|    86.51| 4361| 732.19|
|  Croatia|    20.55|    86.52|   73| 721.31|
|    Italy|    42.15|    88.87|11041|1479.47|
|    Spain|    28.55|    87.36| 6580|1002.11|
|       US|    36.28|    88.99|37726|1273.43|
|  Morocco|     19.7|    88.56|   28| 691.47|
|  Ukraine|     9.25|    83.83|   14| 324.68|
|   Israel|    31.88|    88.51|  500|1118.99|
|   Cyprus|    15.25|     87.0|   11| 535.28|
|  Uruguay|    27.25|    86.72|  109| 956.48|
|   Mexico|     27.9|     85.1|   65| 979.29|
|  Georgia|    19.04|    87.76|   